# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system

In [1]:
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Array{Float64,1}:
 1.47664 
 0.925843
 1.56612 

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA=LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [2]:
Alu = lufact(A)

Base.LinAlg.LU{Float64,Array{Float64,2}} with factors L and U:
[1.0 0.0 0.0; 0.547144 1.0 0.0; 0.507076 -0.0024921 1.0]
[0.992856 0.233381 0.250403; 0.0 0.830578 -0.0723901; 0.0 0.0 0.178965]

In [3]:
typeof(Alu)

Base.LinAlg.LU{Float64,Array{Float64,2}}

In [18]:
Alu

Base.LinAlg.LU{Float64,Array{Float64,2}} with factors L and U:
[1.0 0.0 0.0; 0.547144 1.0 0.0; 0.507076 -0.0024921 1.0]
[0.992856 0.233381 0.250403; 0.0 0.830578 -0.0723901; 0.0 0.0 0.178965]

The different parts of the factorization can be extracted by special indexing

In [4]:
Alu[:P]

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  0.0  1.0
 0.0  1.0  0.0

In [5]:
Alu[:L]

3×3 Array{Float64,2}:
 1.0        0.0        0.0
 0.547144   1.0        0.0
 0.507076  -0.0024921  1.0

In [6]:
Alu[:U]

3×3 Array{Float64,2}:
 0.992856  0.233381   0.250403 
 0.0       0.830578  -0.0723901
 0.0       0.0        0.178965 

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [7]:
A\b

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

In [8]:
Alu\b

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [9]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [10]:
Aqr = qrfact(A)

Base.LinAlg.QRCompactWY{Float64,Array{Float64,2}} with factors Q and R:
[-0.801542 -0.390027 -0.45322; -0.406443 -0.200545 0.891396; -0.438559 0.898699 0.00222145]
[-1.23868 -0.654581 -0.353467; 0.0 0.746855 -0.100984; 0.0 0.0 0.159528]

The matrices `Q` and `R` can be extracted from the QR factorization object via

In [11]:
Aqr[:Q]

3×3 Base.LinAlg.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.801542  -0.390027  -0.45322   
 -0.406443  -0.200545   0.891396  
 -0.438559   0.898699   0.00222145

In [ ]:
Aqr[:R]

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [20]:
Asym = A + A'
AsymEig = eigfact(Asym)

Base.LinAlg.Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}([-1.08604, 0.602601, 2.83093], [-0.0238415 0.615066 0.788115; -0.684949 -0.584283 0.435269; 0.728201 -0.529441 0.435219])

The values and the vectors can be extracted from the Eigen type by special indexing

In [21]:
AsymEig[:values]

3-element Array{Float64,1}:
 -1.08604 
  0.602601
  2.83093 

In [22]:
AsymEig[:vectors]

3×3 Array{Float64,2}:
 -0.0238415   0.615066  0.788115
 -0.684949   -0.584283  0.435269
  0.728201   -0.529441  0.435219

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [23]:
inv(AsymEig)*Asym

3×3 Array{Float64,2}:
  1.0          -1.11022e-15   1.74166e-15
  1.66533e-15   1.0          -4.77396e-15
 -4.996e-16     3.46945e-17   1.0        

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [24]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [25]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [26]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

-1.341535799166017

In [27]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [28]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [29]:
@time eigvals(Asym);

  2.710131 seconds (18.08 k allocations: 8.840 MiB, 0.17% gc time)


In [30]:
@time eigvals(Asym_noisy);

 16.076212 seconds (24 allocations: 7.921 MiB)


In [31]:
@time eigvals(Asym_explicit);

  2.277591 seconds (1.86 k allocations: 8.088 MiB, 0.75% gc time)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [32]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  1.129017 seconds (77.35 k allocations: 187.373 MiB, 16.35% gc time)


6.081576487356633

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [33]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [34]:
Arational = convert(Matrix{Rational{BigInt}}, rand(1:10,3,3))/10

3×3 Array{Rational{BigInt},2}:
 1//5   3//5  1//5 
 1//1   1//5  7//10
 3//10  1//2  9//10

In [35]:
x = fill(1, (3))
b = Arational*x

3-element Array{Rational{BigInt},1}:
  1//1 
 19//10
 17//10

In [36]:
Arational\b

3-element Array{Rational{BigInt},1}:
 1//1
 1//1
 1//1

In [37]:
lufact(Arational)

Base.LinAlg.LU{Rational{BigInt},Array{Rational{BigInt},2}} with factors L and U:
Rational{BigInt}[1//1 0//1 0//1; 5//1 1//1 0//1; 3//2 1//7 1//1]
Rational{BigInt}[1//5 3//5 1//5; 0//1 -14//5 -3//10; 0//1 0//1 9//14]

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```

In [39]:
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]

5×5 Array{Int64,2}:
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36

In [40]:
eigvals(A)

5-element Array{Float64,1}:
 -128.493 
  -55.8878
   42.7522
   87.1611
  542.468 

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [42]:
Diagonal(A)

5×5 Diagonal{Int64}:
 140    ⋅    ⋅   ⋅   ⋅
   ⋅  106    ⋅   ⋅   ⋅
   ⋅    ⋅  152   ⋅   ⋅
   ⋅    ⋅    ⋅  54   ⋅
   ⋅    ⋅    ⋅   ⋅  36

#### 11.3 
Create a `LowerTriangular` matrix from `A`.

In [43]:
LowerTriangular(A)

5×5 LowerTriangular{Int64,Array{Int64,2}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36

### Please let us know how we're doing!
https://tinyurl.com/introJuliaFeedback